In [1]:
import cv2
import os
import numpy as np

class DefectDetectionPipeline:
    def __init__(self, resize_dim=(128, 128), min_area=100):
        """
        Initializes the defect detection pipeline with image resizing dimensions and minimum area for defect detection.
        
        Args:
            resize_dim (tuple): Desired dimensions for resizing the image.
            min_area (int): Minimum area to consider a contour as a defect.
        """
        self.resize_dim = resize_dim
        self.min_area = min_area

    def load_image(self, image_path):
        """
        Load an image, convert it to grayscale, and resize it.
        
        Args:
            image_path (str): Path to the image file.
        
        Returns:
            image: Preprocessed image.
        """
        # Load the image
        image = cv2.imread(image_path)
        
        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Resize the image
        resized_image = cv2.resize(gray_image, self.resize_dim)
        
        return resized_image

    def detect_defects(self, image):
        """
        Detect defects in the image using basic image processing techniques.
        
        Args:
            image: Grayscale image.
        
        Returns:
            defect_image: Image with detected defects highlighted.
            contours: List of contours found in the image.
        """
        # Apply GaussianBlur to smooth the image
        blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
        
        # Apply adaptive thresholding to detect edges
        thresh_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                             cv2.THRESH_BINARY_INV, 11, 2)
        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresh_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Create a copy of the original image to draw contours on
        defect_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        
        # Draw contours (defects) on the image
        cv2.drawContours(defect_image, contours, -1, (0, 0, 255), 2)
        
        return defect_image, contours

    def analyze_defects(self, contours):
        """
        Analyze the detected defects based on contour area.
        
        Args:
            contours: List of contours.
        
        Returns:
            defect_count: Number of detected defects.
            large_defects: Contours that are considered large defects.
        """
        defect_count = 0
        large_defects = []
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > self.min_area:
                defect_count += 1
                large_defects.append(contour)
        
        return defect_count, large_defects

    def defect_detection_pipeline(self, image_path):
        """
        Complete defect detection pipeline.
        
        Args:
            image_path (str): Path to the image file.
        """
        # Load and preprocess the image
        image = self.load_image(image_path)
        
        # Detect defects in the image
        defect_image, contours = self.detect_defects(image)
        
        # Analyze the detected defects
        defect_count, large_defects = self.analyze_defects(contours)
        
        # Display the results
        cv2.imshow("Original Image", image)
        cv2.imshow("Defected Image", defect_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        print(f"Number of significant defects detected: {defect_count}")




In [3]:
# Example usage
if __name__ == "__main__":
    # Instantiate the defect detection pipeline
    defect_detector = DefectDetectionPipeline(resize_dim=(128, 128), min_area=100)
    
    # Test with an image path
    image_path = 'dataset/1234/Oring/test/frame_2023-10-06_11-18-43.png'
    defect_detector.defect_detection_pipeline(image_path)

Number of significant defects detected: 1


In [6]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

class ProductClassifier:
    def __init__(self, cnn_model_path, resize_dim=(128, 128)):
        """
        Initialize the ProductClassifier with the CNN model path and image resize dimensions.

        Args:
            cnn_model_path (str): Path to the trained CNN model.
            resize_dim (tuple): Desired dimensions for resizing the image.
        """
        self.cnn_model = load_model(cnn_model_path)
        self.resize_dim = resize_dim

    def remove_background(self, image):
        """
        Remove the background from the image, leaving only the object.

        Args:
            image: Input image (BGR).

        Returns:
            mask: Binary mask of the object.
            result: Image with the background removed.
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        _, mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        if np.mean(image[mask == 255]) < np.mean(image[mask == 0]):
            mask = cv2.bitwise_not(mask)

        result = cv2.bitwise_and(image, image, mask=mask)
        return mask, result

    def classify_and_display(self, image_path):
        """
        Classify the product in the image and display the results.

        Args:
            image_path: Path to the input image.
        """
        # Load and remove background
        image = cv2.imread(image_path)
        _, object_image = self.remove_background(image)

        # Preprocess the image for classification
        object_image_resized = cv2.resize(object_image, self.resize_dim) / 255.0  # Resize and normalize
        object_image_resized = np.expand_dims(object_image_resized, axis=0)

        # Classify the object image using the CNN
        classification = self.cnn_model.predict(object_image_resized)
        label = np.argmax(classification, axis=1)[0]

        # Determine if the product is "Good" or "Defected"
        product_status = 'Good' if label == 0 else 'Good'

        # Display results
        print(f"{image_path}: Classification - {product_status}")
        cv2.putText(object_image, f"Classification: {product_status}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if product_status == 'Good' else (0, 0, 255), 2)

        cv2.imshow(f"Result - {os.path.basename(image_path)}", object_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def test_directory(self, image_dir):
        """
        Test all images in the directory and classify them as "Good" or "Defected".

        Args:
            image_dir: Path to the directory containing images.
        """
        for image_name in os.listdir(image_dir):
            image_path = os.path.join(image_dir, image_name)
            if os.path.isfile(image_path):
                self.classify_and_display(image_path)




In [25]:
# Example usage:
if __name__ == "__main__":
    # Load the trained CNN model
    cnn_model_path = 'cnn_model.h5'  # Replace with your model path
    classifier = ProductClassifier(cnn_model_path)

    # Specify the directory containing the images
    image_dir = 'dataset/dbottle/test/'  # Replace with your directory path

    # Run the test on the whole directory
    classifier.test_directory(image_dir)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
MTVAD/dbottle/test/broken_large_000.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MTVAD/dbottle/test/broken_large_001.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
MTVAD/dbottle/test/broken_large_002.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
MTVAD/dbottle/test/broken_large_003.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MTVAD/dbottle/test/broken_large_004.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MTVAD/dbottle/test/broken_large_005.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MTVAD/dbottle/test/broken_large_006.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
MTVAD/dbottle/test/broken_large_007.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
MTVAD/dbottle/test/broken_large_008.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
MTVAD/dbottle/te

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
MTVAD/dbottle/test/contamination_017.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
MTVAD/dbottle/test/contamination_018.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
MTVAD/dbottle/test/contamination_019.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
MTVAD/dbottle/test/contamination_020.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
MTVAD/dbottle/test/good_000.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
MTVAD/dbottle/test/good_001.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
MTVAD/dbottle/test/good_002.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
MTVAD/dbottle/test/good_003.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
MTVAD/dbottle/test/good_004.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
MTVAD/dbottle/test/good_005.png: Classification - Def

In [26]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

class AdvancedProductClassifier:
    def __init__(self, cnn_model_path, resize_dim=(128, 128)):
        """
        Initialize the AdvancedProductClassifier with the CNN model path and image resize dimensions.

        Args:
            cnn_model_path (str): Path to the trained CNN model.
            resize_dim (tuple): Desired dimensions for resizing the image.
        """
        self.cnn_model = load_model(cnn_model_path)
        self.resize_dim = resize_dim

    def remove_background(self, image):
        """
        Remove the background from the image, leaving only the object.

        Args:
            image: Input image (BGR).

        Returns:
            mask: Binary mask of the object.
            result: Image with the background removed.
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        _, mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        
        if np.mean(image[mask == 255]) < np.mean(image[mask == 0]):
            mask = cv2.bitwise_not(mask)
        
        result = cv2.bitwise_and(image, image, mask=mask)
        
        return mask, result

    def classify_and_display(self, image_path):
        """
        Classify the product in the image and display the results.

        Args:
            image_path: Path to the input image.
        """
        # Load and remove background
        image = cv2.imread(image_path)
        _, object_image = self.remove_background(image)
        
        # Preprocess the image for classification
        object_image_resized = cv2.resize(object_image, self.resize_dim) / 255.0  # Resize and normalize
        object_image_resized = np.expand_dims(object_image_resized, axis=0)
        
        # Classify the object image using the CNN
        classification = self.cnn_model.predict(object_image_resized)
        label = np.argmax(classification, axis=1)[0]
        
        # Determine if the product is "Good" or "Defected"
        product_status = 'Good' if label == 0 else 'Defected'
        
        # Display results
        print(f"{image_path}: Classification - {product_status}")
        cv2.putText(object_image, f"Classification: {product_status}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if product_status == 'Good' else (0, 0, 255), 2)
        
        cv2.imshow(f"Result - {os.path.basename(image_path)}", object_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def test_directory(self, image_dir):

        for image_name in os.listdir(image_dir):
            image_path = os.path.join(image_dir, image_name)
            if os.path.isfile(image_path):
                self.classify_and_display(image_path)



In [27]:
# Example usage:
if __name__ == "__main__":
    # Load the trained CNN model
    cnn_model_path = 'cnn_model.h5'  # Replace with your model path
    classifier = AdvancedProductClassifier(cnn_model_path)

    # Specify the directory containing the images
    image_dir = 'MTVAD/dbottle/test/'  # Replace with your directory path

    # Run the test on the whole directory
    classifier.test_directory(image_dir)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
MTVAD/dbottle/test/broken_large_000.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
MTVAD/dbottle/test/broken_large_001.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MTVAD/dbottle/test/broken_large_002.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
MTVAD/dbottle/test/broken_large_003.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MTVAD/dbottle/test/broken_large_004.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
MTVAD/dbottle/test/broken_large_005.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MTVAD/dbottle/test/broken_large_006.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
MTVAD/dbottle/test/broken_large_007.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
MTVAD/dbottle/test/broken_large_008.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MTVAD/dbottle/tes

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
MTVAD/dbottle/test/contamination_017.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
MTVAD/dbottle/test/contamination_018.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
MTVAD/dbottle/test/contamination_019.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
MTVAD/dbottle/test/contamination_020.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
MTVAD/dbottle/test/good_000.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
MTVAD/dbottle/test/good_001.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
MTVAD/dbottle/test/good_002.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
MTVAD/dbottle/test/good_003.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
MTVAD/dbottle/test/good_004.png: Classification - Defected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
MTVAD/dbottle/test/good_005.png: Classification - Def

In [4]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator

class DefectDetection:
    def __init__(self, target_size=(128, 128)):
        """
        Initialize the DefectDetection with the target image size.

        Args:
            target_size (tuple): Desired dimensions for resizing the images.
        """
        self.target_size = target_size

    def load_images_from_directory(self, directory):
        """
        Load images from a directory, resize them, and return as numpy array.

        Args:
            directory: Path to the directory containing images.

        Returns:
            X: Array of images.
        """
        images = []
        for image_name in os.listdir(directory):
            image_path = os.path.join(directory, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, self.target_size)
                image = img_to_array(image) / 255.0  # Normalize the image
                images.append(image)
        return np.array(images)

    def build_cnn_model(self, input_shape):
        """
        Build and compile a CNN model.

        Args:
            input_shape: Shape of the input images.

        Returns:
            model: Compiled CNN model.
        """
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))  # Binary classification: Good or Defected

        model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def train_on_good_images(self, good_images_dir, model_save_path, epochs=10, batch_size=32):
        """
        Train the CNN model on good images.

        Args:
            good_images_dir: Path to the directory containing good images.
            model_save_path: Path to save the trained model.
            epochs: Number of training epochs.
            batch_size: Batch size for training.
        """
        good_images = self.load_images_from_directory(good_images_dir)
        good_labels = np.zeros((good_images.shape[0], 1))  # All labels are 0 (Good)
        
        model = self.build_cnn_model((self.target_size[0], self.target_size[1], 3))
        
        # Using data augmentation to increase variety
        datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, 
                                     shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
        
        model.fit(datagen.flow(good_images, good_labels, batch_size=batch_size), epochs=epochs)
        
        model.save(model_save_path)
        print(f"Model trained and saved to {model_save_path}")

    def classify_test_images(self, test_images_dir, model_path):
        """
        Classify images in a directory as Good or Defected.

        Args:
            test_images_dir: Path to the directory containing test images.
            model_path: Path to the trained CNN model.
        """
        model = load_model(model_path)
        
        for image_name in os.listdir(test_images_dir):
            image_path = os.path.join(test_images_dir, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                # Preprocess the image
                image_resized = cv2.resize(image, self.target_size)
                image_resized = img_to_array(image_resized) / 255.0
                image_resized = np.expand_dims(image_resized, axis=0)
                
                # Classify the image
                prediction = model.predict(image_resized)[0][0]
                label = 'Good' if prediction < 0.5 else 'Defected'
                
                print(f"{image_name}: {label}")
                cv2.putText(image, f"Classification: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                            (0, 255, 0) if label == 'Good' else (0, 0, 255), 2)
                
                cv2.imshow(f"Result - {image_name}", image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

# Example usage:
if __name__ == "__main__":
    good_images_dir = 'dataset/MTVAD/dbottle/test/'
    test_images_dir = 'dataset/MTVAD/dbottle/train/'
    model_save_path = 'test1_model.h5'

    classifier = DefectDetection()

    # Step 1: Train the model on good images
    classifier.train_on_good_images(good_images_dir, model_save_path, epochs=10)

    # Step 2: Test the model on test images
    classifier.classify_test_images(test_images_dir, model_save_path)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 247ms/step - accuracy: 0.7516 - loss: 0.3914
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step - accuracy: 1.0000 - loss: 3.5660e-08
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step - accuracy: 1.0000 - loss: 4.7411e-15
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 218ms/step - accuracy: 1.0000 - loss: 9.2989e-21
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step - accuracy: 1.0000 - loss: 8.3611e-27
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 225ms/step - accuracy: 1.0000 - loss: 1.2421e-26
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 223ms/step - accuracy: 1.0000 - loss: 1.2446e-22
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 235ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 337ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 347ms/step - accuracy: 1.0000 - loss: 0.0000e+00


Model trained and saved to test1_model.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
good_000.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
good_001.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
good_002.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
good_003.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
good_004.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
good_005.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
good_006.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
good_007.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
good_008.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
good_009.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
good_010.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
good_011.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
good_012.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
good_013.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
good_014.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
good_015.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
good_016.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
good_180.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
good_181.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
good_182.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
good_183.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
good_184.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
good_185.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
good_186.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
good_187.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
good_188.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
good_189.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
good_190.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
good_191.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
good_192.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
good_193.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
good_194.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
good_195.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
good_196.png: Good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67m

In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split

class DefectDetection:
    def __init__(self, target_size=(128, 128)):
        """
        Initialize the DefectDetection with the target image size.

        Args:
            target_size (tuple): Desired dimensions for resizing the images.
        """
        self.target_size = target_size

    def load_images_from_directory(self, directory, label):
        """
        Load images from a directory, resize them, and return as numpy array along with their labels.

        Args:
            directory: Path to the directory containing images.
            label: Label to assign to the loaded images (0 for "Good", 1 for "Defected").

        Returns:
            X: Array of images.
            y: Array of corresponding labels.
        """
        images = []
        labels = []
        for image_name in os.listdir(directory):
            image_path = os.path.join(directory, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, self.target_size)
                image = img_to_array(image) / 255.0  # Normalize the image
                images.append(image)
                labels.append(label)
            else:
                print(f"Warning: Failed to load image {image_name}")

        return np.array(images), np.array(labels)

    def build_cnn_model(self, input_shape):
        """
        Build and compile a CNN model.

        Args:
            input_shape: Shape of the input images.

        Returns:
            model: Compiled CNN model.
        """
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))  # Binary classification: Good or Defected

        model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def train_on_images(self, good_images_dir, defected_images_dir, model_save_path, epochs=10, batch_size=32):
        """
        Train the CNN model on both good and defected images.

        Args:
            good_images_dir: Path to the directory containing good images.
            defected_images_dir: Path to the directory containing defected images.
            model_save_path: Path to save the trained model.
            epochs: Number of training epochs.
            batch_size: Batch size for training.
        """
        # Load good and defected images
        good_images, good_labels = self.load_images_from_directory(good_images_dir, label=0)
        defected_images, defected_labels = self.load_images_from_directory(defected_images_dir, label=1)

        # Combine the data
        X = np.concatenate([good_images, defected_images], axis=0)
        y = np.concatenate([good_labels, defected_labels], axis=0)

        # Split the data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

        model = self.build_cnn_model((self.target_size[0], self.target_size[1], 3))

        # Using data augmentation to increase variety
        datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, 
                                     shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

        # Train the model with validation
        model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                  validation_data=(X_val, y_val), epochs=epochs)

        model.save(model_save_path)
        print(f"Model trained and saved to {model_save_path}")

    def classify_test_images(self, test_images_dir, model_path):
        """
        Classify images in a directory as Good or Defected.

        Args:
            test_images_dir: Path to the directory containing test images.
            model_path: Path to the trained CNN model.
        """
        model = load_model(model_path)
        print("Model loaded successfully.")

        for image_name in os.listdir(test_images_dir):
            print(f"Processing {image_name}")
            image_path = os.path.join(test_images_dir, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                print(f"Image loaded: {image_name}")
                # Preprocess the image
                image_resized = cv2.resize(image, self.target_size)
                image_resized = img_to_array(image_resized) / 255.0
                image_resized = np.expand_dims(image_resized, axis=0)

                # Classify the image
                prediction = model.predict(image_resized)[0][0]
                label = 'Good' if prediction < 0.5 else 'Defected'

                print(f"Prediction for {image_name}: {label}")
                cv2.putText(image, f"Classification: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                            (0, 255, 0) if label == 'Good' else (0, 0, 255), 2)

                cv2.imshow(f"Result - {image_name}", image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            else:
                print(f"Warning: Failed to load image {image_name}")

# Example usage:
if __name__ == "__main__":
    good_images_dir = 'dataset/1234/Oring/test/'
    defected_images_dir = 'dataset/1234/Oring/train/'
    test_images_dir = 'dataset/1234/Oring/test/'
    model_save_path = 'defect_detection_model.h5'

    classifier = DefectDetection()

    # Step 1: Train the model on both good and defected images
    classifier.train_on_images(good_images_dir, defected_images_dir, model_save_path, epochs=10)

    # Step 2: Test the model on test images
    classifier.classify_test_images(test_images_dir, model_save_path)


C:\Users\Dhiraj-Pc\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


C:\Users\Dhiraj-Pc\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 474ms/step - accuracy: 0.5098 - loss: 0.6935 - val_accuracy: 0.8333 - val_loss: 0.6612
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 353ms/step - accuracy: 0.5894 - loss: 0.7032 - val_accuracy: 0.5417 - val_loss: 0.6226
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 332ms/step - accuracy: 0.6045 - loss: 0.6456 - val_accuracy: 0.5417 - val_loss: 0.5693
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 324ms/step - accuracy: 0.6728 - loss: 0.6253 - val_accuracy: 0.9583 - val_loss: 0.5499
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 325ms/step - accuracy: 0.7808 - loss: 0.5887 - val_accuracy: 0.9583 - val_loss: 0.4412
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 326ms/step - accuracy: 0.8518 - loss: 0.5053 - val_accuracy: 0.9167 - val_loss: 0.3045
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 330ms/step - accuracy: 0.8568 - loss: 0.4265 - val_accuracy: 0.9583 - val_loss: 0.2422
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 345ms/step - accuracy: 0.8265 - loss: 0.4554 - val_accuracy: 0.9583 - val_loss: 0.2601
Epo

Model trained and saved to defect_detection_model.h5


Model loaded successfully.
Processing frame_2023-10-06_11-18-43.png
Image loaded: frame_2023-10-06_11-18-43.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Prediction for frame_2023-10-06_11-18-43.png: Defected
Processing frame_2023-10-06_11-18-44 - Copy.png
Image loaded: frame_2023-10-06_11-18-44 - Copy.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction for frame_2023-10-06_11-18-44 - Copy.png: Defected
Processing frame_2023-10-06_11-18-44.png
Image loaded: frame_2023-10-06_11-18-44.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction for frame_2023-10-06_11-18-44.png: Defected
Processing frame_2023-10-06_11-19-03 - Copy - Copy (2).png
Image loaded: frame_2023-10-06_11-19-03 - Copy - Copy (2).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction for frame_2023-10-06_11-19-03 - Copy - Copy (2).png: Good
Processing frame_2023-10-06_11-19-03 - Copy - Copy.png
Image loaded: frame_2023-10-06_11-19-03 - Copy - Copy.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction for frame_2023-10-06_11-19-

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Prediction for frame_2023-10-17_10-42-26 - Copy (27) - Copy.png: Good
Processing frame_2023-10-17_10-42-26 - Copy (28) - Copy.png
Image loaded: frame_2023-10-17_10-42-26 - Copy (28) - Copy.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction for frame_2023-10-17_10-42-26 - Copy (28) - Copy.png: Good
Processing frame_2023-10-17_10-42-26 - Copy (29) - Copy.png
Image loaded: frame_2023-10-17_10-42-26 - Copy (29) - Copy.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction for frame_2023-10-17_10-42-26 - Copy (29) - Copy.png: Good
Processing frame_2023-10-17_10-42-26 - Copy (3) - Copy.png
Image loaded: frame_2023-10-17_10-42-26 - Copy (3) - Copy.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction for frame_2023-10-17_10-42-26 - Copy (3) - Copy.png: Good
Processing frame_2023-10-17_10-42-26 - Copy (3).png
Image loaded: frame_2023-10-17_10-42-26 - Copy (3).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction for frame_2023-10-17_10-42-26 - Copy (3).png: 